In [ ]:
import pandas as pd

ecom_data = pd.read_csv(r"data\online_retail.csv")

print(ecom_data.isnull().sum())


In [ ]:
ecom = ecom_data.dropna(subset=["Description"])

ecom_customers= ecom.copy()
ecom_customers["Revenue"] = ecom_customers["Quantity"]*ecom_customers["UnitPrice"]

sales = ecom_customers[ (ecom_customers["Quantity"] > 0) & (ecom_customers["UnitPrice"]>0) &(~ecom_customers["InvoiceNo"].str.startswith("C"))].copy()
returns = ecom[ (ecom["Quantity"] < 0) | (ecom["UnitPrice"]<0) | (ecom["InvoiceNo"].str.startswith("C")) ].copy()


sales["InvoiceDate"]= pd.to_datetime(sales["InvoiceDate"])
sales["Month"] = sales["InvoiceDate"].dt.to_period("M")
sales.head()


In [ ]:
#monthly analysis
monthly_orders = sales.groupby("Month")["Revenue"].sum().reset_index(name="Revenue")
import matplotlib.pyplot as plt
plt.figure(figsize=(7,5))
plt.bar(monthly_orders["Month"].astype(str), monthly_orders["Revenue"]/1000000)
plt.title("Monthly Revenue analysis")
plt.xlabel("Months")
plt.ylabel("Revenue (euro, in Million)")
plt.xticks(rotation=45)
plt.tight_layout()

monthly_orders.head(10)

In [ ]:
# top 10 country wise analysis

country_wise = sales.groupby("Country")["Revenue"].sum().sort_values(ascending=False).reset_index()
country=country_wise.head(10).Country.to_numpy()
revi = country_wise.head(10).Revenue.astype(int).to_numpy()
plt.figure(figsize=(7,5))
plt.barh(country, revi)
plt.show()

In [ ]:
# top 5 country wose analysis

country_wise = sales.groupby("Country")["Revenue"].sum().sort_values(ascending=False).reset_index()

colors = ['lightgreen', 'yellow', 'pink', 'skyblue', "red", "grey"]
country=country_wise.head(5).Country.to_numpy()
pi = country_wise.head(5).Revenue.astype(int).to_numpy()
print(pi)
plt.figure(figsize=(5,5))
plt.pie(pi, colors=colors, labels=country)
plt.tight_layout()
plt.show()

In [ ]:
sales["Revenue_M"] = country_wise["Revenue"]/1000000


In [ ]:
# top customers

top_customers = sales.groupby("CustomerID")["Revenue"].sum().sort_values(ascending=False).reset_index(name="Revenue")
t10=top_customers.head(10).copy()

# top customer preferences

top_ids=t10["CustomerID"]

top_sales = sales[sales["CustomerID"].isin(top_ids)]
top_sales.head()


In [ ]:
# most sold items:

top_items= sales.groupby("Description")["Revenue"].sum().sort_values(ascending=False).reset_index()
top10it=top_items.head(10)

plt.figure(figsize=(9,7))
plt.barh(top10it["Description"], top10it["Revenue"])
plt.xlabel("Revenue (in euro)")
plt.ylabel("items purchased")
plt.xticks(rotation=60)
plt.title("Top Items sold")
plt.show()

In [ ]:
# return anakysis...
returns["Revenue"] = returns["Quantity"]* returns["UnitPrice"]
returns["Revenue"] = returns["Revenue"].abs()
returned_products = returns.groupby("Description")["Revenue"].sum().sort_values(ascending=False).reset_index().head(6)
returned_products

plt.figure(figsize=(8,6))
plt.barh(returned_products["Description"], returned_products["Revenue"])
plt.ylabel("Retruned products")
plt.xlabel("Return revenue (in euro)")
plt.title("Returned items vs Returned revenue")
plt.tight_layout()
plt.show()

In [ ]:
# KPI (Key performance index)

Total_orders = sales["InvoiceNo"].nunique()
Total_customers = sales["CustomerID"].nunique()
Total_revenue = sales["Revenue"].sum()
total_return_revenue = returns["Revenue"].sum()
average_order_value = Total_revenue/Total_orders
return_rate = total_return_revenue/Total_revenue

print("===== KEY PERFORMANCE INDICES =====")
print(f"Total Orders: {Total_orders}")
print(f"Total customers: {Total_customers}")
print(f"Total revenue: {Total_revenue}")
print(f"Total return revenue: {total_return_revenue}")
print(f"Average order value: {average_order_value}")
print(f"Return rate: {return_rate}")

In [ ]:
sales.to_csv("sales_cleaned_dataset", index=False)
returns.to_csv("Returns_dataset", index=False)